## Hyperparameter Optimization 

Hyperparameters (HP) are those parameters that machine or deep learning algorithms cannot learn by training. The value of those parameters needs to be set before the training process and it can control how the algorithm learns from the data. Hyperparameters (HP) can be the number of layers or a neural network or number of nodes in a layer, it can be the learning rate or the type of optimizer that the neural network can use. Getting the right hyperparameters can be counter-intuitive, it is more of a trial and error process. Hence, we have the need to find a way to automatically tune those parameters to get the best combination that will maximize the ML model performance.

###Objective Function:

Machine Learning can be simply defined as learning behaviour from experience. By learning here, we mean getting better and improving at some certain task over time. But what constitutes an improvement?

In order to develop a function to judge the improvement, we need to have formal measures of how good or bad our models are. In machine learning, we call these “objective functions”. The most common objective function is squared error which is basically the difference between the predicted value and the actual ground truth. If this value is large, then the accuracy is low. Our target from optimizing the hyperparameters will be to minimize this objective function. Getting the right combination of the model’s hyperparameters can contribute largely into the model’s performance, Let’s have a look at how we can tune those parameters to minimize the objective function:

## 1- Grid Search:

![Search Grid](https://raw.githubusercontent.com/quantumhack/data/40822efd2f3467e6985909c96b5501313fcb5d46/Projects/Grid.png)

One way of tuning the hyperparameters is to define an equal range for each HP and have the computer try all possible combinations of parameter values. It can be a great approach to get the best combination of HPs. It works by putting the HPs that you want to tune in an n-dimensional grid then try each and every combination in this grid. Let’s have a look at how it works, the example below will try to find the best combination of hyperparameters to build a classifier for the MNIST dataset.

The code above will brute force and try every possible combination of those three hyper-parameters then print out the best combination.

In [0]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

In [0]:
#load the MNIST dataset and normalize it
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

#we are going to evaluate the number of hidden layers (3 cases) and 
#hidden units (3 cases)
hidden_layers_ = [1, 2, 3]
hidden_units_ = [5, 10, 15]
#also we are going to evaluate the best solver (2 cases) and the 
#learning rate (3 cases)
solver_ = ['sgd', 'adam']
lr_ = [0.001, 0.01, 0.1]

#we set some constant values which could also be tunned but for the sake of 
#making the execution shorter we do not
epochs_ = 10
batch_size_ = 32
momentum_ = 0.9	
nesterov_ = True

#set some constants for the algorithm
test_loss = [0] * (len(hidden_units_) * len(hidden_layers_) * len(solver_) * len(lr_))
test_acc =  [0] * (len(hidden_units_) * len(hidden_layers_) * len(solver_) * len(lr_))
caso = 0
max_acc = 0

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
#we run the algorithms for every combination of every case and later we select
#the one with the best accuracy

for i in range(len(hidden_units_)):
  for j in range(len(hidden_layers_)):
    for k in range(len(lr_)):
      for l in range(len(solver_)):
        model = Sequential()
        model.add(layers.Flatten(input_shape=(28, 28)))
        model.add(layers.Dense(hidden_units_[i], input_shape=(28, 28), activation='relu'))
        for i in range(hidden_layers_[j] - 1):
          model.add(layers.Dense(hidden_units_[i], activation='relu'))
        model.add(layers.Dense(10,activation='softmax'))
        
        if solver_[l] == 'sgd':
          optimizer = optimizers.SGD(lr=lr_[k], momentum = momentum_, nesterov=nesterov_)
        if solver_[l] == 'Adam':
          optimizer = optimizers.Adam(lr=lr_[k])

        model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        history = model.fit(x_train, y_train, epochs=epochs_, verbose=0, batch_size=batch_size_, validation_data=(x_test, y_test))
        test_loss[caso], test_acc[caso] = model.evaluate(x_test, y_test, verbose=0)  
        if test_acc[caso]>max_acc:
          max_acc = test_acc[caso]
          neu_opt = hidden_units_[i]
          lay_opt = hidden_layers_[j]
          lra_opt = lr_[k]
          sol_opt = solver_[l]
          hist_opt = history
        
        caso += 1

In [0]:
#then we print the accuracy for every combination we tried
#and later print the best case
caso = 0
for i in range(len(hidden_units_)):
	for j in range(len(hidden_layers_)):
		for k in range(len(lr_)):
			for l in range(len(solver_)):
				print("Cantidad de layers",hidden_layers_[j],". Cantidad de neuronas", hidden_units_[i],". Solver:",solver_[l],". Learning rate:", lr_[k],". Accuracy:", test_acc[caso])
				caso += 1				
print("---------------------------------------------")
print("El Accuracy máximo es:",max_acc,". Para el caso con",lay_opt," layers,",neu_opt,"cantidad de neuronas, ",sol_opt," y un learning rate de",lra_opt)

Cantidad de layers 1 . Cantidad de neuronas 5 . Solver: sgd . Learning rate: 0.001 . Accuracy: 0.8885999917984009
Cantidad de layers 1 . Cantidad de neuronas 5 . Solver: adam . Learning rate: 0.001 . Accuracy: 0.8787000179290771
Cantidad de layers 1 . Cantidad de neuronas 5 . Solver: sgd . Learning rate: 0.01 . Accuracy: 0.8532999753952026
Cantidad de layers 1 . Cantidad de neuronas 5 . Solver: adam . Learning rate: 0.01 . Accuracy: 0.8877000212669373
Cantidad de layers 1 . Cantidad de neuronas 5 . Solver: sgd . Learning rate: 0.1 . Accuracy: 0.7152000069618225
Cantidad de layers 1 . Cantidad de neuronas 5 . Solver: adam . Learning rate: 0.1 . Accuracy: 0.7678999900817871
Cantidad de layers 2 . Cantidad de neuronas 5 . Solver: sgd . Learning rate: 0.001 . Accuracy: 0.8863999843597412
Cantidad de layers 2 . Cantidad de neuronas 5 . Solver: adam . Learning rate: 0.001 . Accuracy: 0.8636999726295471
Cantidad de layers 2 . Cantidad de neuronas 5 . Solver: sgd . Learning rate: 0.01 . Accura

## 2- Bayesian Optimization:

![Bayesian Optimization:](https://raw.githubusercontent.com/quantumhack/data/40822efd2f3467e6985909c96b5501313fcb5d46/Projects/Bayes.png)

In the above examples, we had an objective metric which is the accuracy and we also had an objective function that tries to maximize the metric and minimize the loss. Bayesian Optimization approach tries to find the value that minimizes an objective function by building a probability model based on past evaluation results of the objective metric. It tries to find the balance between the exploration of the best region that contains the best hyperparameters and the exploitation of this region to maximize/minimize the objective metric.

The problem of optimizing hyper-parameters is that it is an expensive process to assess the performance of a set of HPs because we have to build the corresponding graphs or neural networks in each iteration, then we have to train it, and finally, we have to assess the performance. The optimization process can take hours or days but in this example, we will train on only 10 epochs.

Let’s have a look at the example below that uses Bayesian Optimization to tune a neural network for the MNIST dataset. We will use scikit-optimize library to perform the HPO. It’s one of the libraries that has implemented the Bayesian Optimization algorithm:

In [0]:
!pip install scikit-optimize

     |████████████████████████████████| 81kB 2.1MB/s 


In [0]:
import keras
import tensorflow
import skopt
import pandas as pd

from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, SGD
from tensorflow.python.keras import backend as K
from tensorflow.python.framework import ops
from skopt import gp_minimize
from skopt.utils import use_named_args
from skopt.space import Real, Categorical, Integer

Using TensorFlow backend.


In [0]:
#we load the dataset, normalize and converto the y outout to categorical
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train/ 255
X_test = X_test/ 255
X_train = X_train.reshape(60000,784)
X_test = X_test.reshape(10000,784)
input_shape= X_train[0].shape
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

#we are going to evaluate the same parameters that we evaluate in the case before
# number of layer, number of units, optimizer and learning rate
dim_num_dense_layers = Integer(low=0, high=2, name='num_dense_layers')
dim_num_dense_nodes = Integer(low=5, high=15, name='num_dense_nodes')
dim_optimizer = Categorical(categories=['Adam', 'SGD'], name='optimizer')
dim_learning_rate = Real(low=1e-3, high=1e-1, prior='log-uniform', name='learning_rate')

#we set the values for the bayesian optimizer to train the neural network
dimensions = [dim_learning_rate,
              dim_num_dense_layers,
              dim_num_dense_nodes,
              dim_optimizer]
default_parameters = [1e-3, 2, 13, 'SGD']

In [0]:
#let's define the model depending on the values for each call
def create_model(learning_rate, num_dense_layers, num_dense_nodes, optimizer):
    model = Sequential()
    model.add(Dense(num_dense_nodes, input_shape= input_shape, activation='relu'))
    for i in range(num_dense_layers):
        name = 'layer_dense_{0}'.format(i+1)
        model.add(Dense(num_dense_nodes, activation='relu', name=name))
    model.add(Dense(10,activation='softmax'))
    if optimizer == 'Adam':
      optimizer_final = Adam(lr=learning_rate)
    if optimizer == 'SGD':
      optimizer_final = SGD(lr=learning_rate)
    model.compile(optimizer=optimizer_final, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

#let's define the fitness depending of the values for each call
@use_named_args(dimensions=dimensions)
def fitness(learning_rate, num_dense_layers, num_dense_nodes, optimizer):

    model = create_model(learning_rate=learning_rate,
                         num_dense_layers=num_dense_layers,
                         num_dense_nodes=num_dense_nodes,
                         optimizer=optimizer)    

    blackbox = model.fit(x=X_train, y=y_train, epochs=10, batch_size=32, validation_split=0.15,)
    accuracy = blackbox.history['val_accuracy'][-1]
    print("Accuracy: {0:.2%}".format(accuracy))

    del model
    K.clear_session()
    ops.reset_default_graph()    

    return -accuracy

In [0]:
K.clear_session()
ops.reset_default_graph()

#call the bayesian optimizer
gp_result = gp_minimize(func=fitness, dimensions=dimensions, n_calls=12, n_jobs=-1, kappa = 5, x0=default_parameters)

Train on 51000 samples, validate on 9000 samples
Epoch 1/10
51000/51000 [==============================] - 3s 51us/step - loss: 2.2942 - accuracy: 0.1146 - val_loss: 2.2701 - val_accuracy: 0.1523
Epoch 2/10
51000/51000 [==============================] - 2s 49us/step - loss: 2.2145 - accuracy: 0.2006 - val_loss: 2.1361 - val_accuracy: 0.2864
Epoch 3/10
51000/51000 [==============================] - 2s 48us/step - loss: 2.0350 - accuracy: 0.3131 - val_loss: 1.9064 - val_accuracy: 0.3551
Epoch 4/10
51000/51000 [==============================] - 2s 48us/step - loss: 1.7881 - accuracy: 0.4057 - val_loss: 1.6322 - val_accuracy: 0.4699
Epoch 5/10
51000/51000 [==============================] - 2s 48us/step - loss: 1.5326 - accuracy: 0.5096 - val_loss: 1.3870 - val_accuracy: 0.5906
Epoch 6/10
51000/51000 [==============================] - 2s 47us/step - loss: 1.3214 - accuracy: 0.6203 - val_loss: 1.1834 - val_accuracy: 0.6858
Epoch 7/10
51000/51000 [==============================] - 2s 49us/ste

In [0]:
#let's see the best accuracy
print("best accuracy was " + str(round(gp_result.fun *-100,2))+"%.")
#and let's see the best values for: learning rate, number of layer
#, number of hidden units and solver

gp_result.x

best accuracy was 94.93%.


[0.09715134469818455, 1, 15, 'SGD']

In [0]:
#finally let's train the neural network with the optimal values and evaluate
#the model on the test set
gp_model = create_model(gp_result.x[0],gp_result.x[1],gp_result.x[2],gp_result.x[3])
gp_model.fit(X_train,y_train, batch_size=10, epochs=10)
gp_model.evaluate(X_test,y_test)

Epoch 1/10
60000/60000 [==============================] - 7s 109us/step - loss: 0.3775 - accuracy: 0.8861
Epoch 2/10
60000/60000 [==============================] - 6s 107us/step - loss: 0.2461 - accuracy: 0.9270
Epoch 3/10
60000/60000 [==============================] - 7s 111us/step - loss: 0.2157 - accuracy: 0.9364
Epoch 4/10
60000/60000 [==============================] - 6s 108us/step - loss: 0.1987 - accuracy: 0.9413
Epoch 5/10
60000/60000 [==============================] - 6s 108us/step - loss: 0.1866 - accuracy: 0.9446
Epoch 6/10
60000/60000 [==============================] - 7s 109us/step - loss: 0.1786 - accuracy: 0.9474
Epoch 7/10
60000/60000 [==============================] - 6s 107us/step - loss: 0.1746 - accuracy: 0.9485
Epoch 8/10
60000/60000 [==============================] - 8s 131us/step - loss: 0.1685 - accuracy: 0.9512
Epoch 9/10
60000/60000 [==============================] - 7s 120us/step - loss: 0.1633 - accuracy: 0.9525
Epoch 10/10
10000/10000 [=====================

[0.16214132775301113, 0.9524999856948853]

## 3 - Genetic Algoritms

![Genetic Algorithms](https://miro.medium.com/max/638/1*exOoxtPvTQpCDcxg52bIWw.jpeg)

The hyperparameters tunning problem is an optimization problem, another technique we can use to solve this is using genetic algorithms. We start by selecting a set of random combinations of hyperparameters and we let them evolve following some rules and constraits to finlly converge to the optimal value.

For doing this we use the DEAP library.

In [0]:
!pip install deap

     |████████████████████████████████| 163kB 2.9MB/s 


In [0]:
from deap import base
from deap import creator
from deap import tools

import matplotlib.pyplot as plt
import random
import numpy
import elitism

from tensorflow.keras.datasets import mnist
from sklearn import model_selection
from sklearn.neural_network import MLPClassifier
from sklearn.exceptions import ConvergenceWarning
from sklearn.utils.testing import ignore_warnings
from math import floor

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
#we start by creating a class with some function of interest for:
#initializing the input with the MNIST dataset,
#codify the hyperparameters into a form understood for the genetic algorithm,
#calculate the accuracy of the model for a certain combination of hyperparameters.
class MlpHyperparametersTest:
    NUM_FOLDS = 5
    def __init__(self, randomSeed):
        self.randomSeed = randomSeed
        self.initDataset()
        self.kfold = model_selection.KFold(n_splits=self.NUM_FOLDS, random_state=self.randomSeed)

    def initDataset(self):     
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        self.X = numpy.vstack((x_train,x_test))
        self.X = self.X / 255.0
        self.X = self.X.reshape(self.X.shape[0], -1)
        y_train = y_train.reshape(-1, 1)
        y_test = y_test.reshape(-1, 1)
        self.y = numpy.vstack((y_train,y_test))        
        self.y = self.y.flatten()

    def convertParams(self, params):
        if round(params[1]) <= 0:
            hiddenLayerSizes = round(params[0]),
        elif round(params[2]) <= 0:
            hiddenLayerSizes = (round(params[0]), round(params[1]))
        elif round(params[3]) <= 0:
            hiddenLayerSizes = (round(params[0]), round(params[1]), round(params[2]))
        else:
            hiddenLayerSizes = (round(params[0]), round(params[1]), round(params[2]), round(params[3]))
        learning_rate_init = numpy.power(10, params[4])
        solver = ['sgd', 'adam'][floor(params[5])]        
        return hiddenLayerSizes, learning_rate_init, solver
        
    @ignore_warnings(category=ConvergenceWarning)
    def getAccuracy(self, params):
        hiddenLayerSizes, learning_rate_init, solver = self.convertParams(params)
        self.classifier = MLPClassifier(random_state=self.randomSeed,
                                        hidden_layer_sizes=hiddenLayerSizes,
                                        activation='relu',
                                        solver=solver,
                                        learning_rate_init=learning_rate_init,
                                        max_iter = 10)
        cv_results = model_selection.cross_val_score(self.classifier,
                                                     self.X,
                                                     self.y,
                                                     cv=self.kfold,
                                                     scoring='accuracy')
        return cv_results.mean()

    def formatParams(self, params):
        hiddenLayerSizes, learning_rate_init, solver = self.convertParams(params)
        return "'hidden_layer_sizes'={}\n " \
               "'solver'='{}'\n " \
               "'learning_rate_init'='{}'\n " \
            .format(hiddenLayerSizes, solver, learning_rate_init)

In [0]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
#set the boundaries for the genetic code
BOUNDS_LOW =  [ 5,  -5, -10, -20, -3, 0]
BOUNDS_HIGH = [15,  15,  15,  15, -1, 1.999]
NUM_OF_PARAMS = len(BOUNDS_HIGH)
#set some constant for the genetic algorithm
POPULATION_SIZE = 7
P_CROSSOVER = 0.8
P_MUTATION = 0.2 
MAX_GENERATIONS = 3
HALL_OF_FAME_SIZE = 2
CROWDING_FACTOR = 10.0

In [0]:
#Here we define the characteristics algorithms and use the constant to 
#set the hyperparameters of the genetic algorithm (we are not tunning thoses lol!)

def classificationAccuracy(individual):
    return test.getAccuracy(individual),

test = MlpHyperparametersTest(RANDOM_SEED)
toolbox = base.Toolbox()
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
for i in range(NUM_OF_PARAMS):
    toolbox.register("attribute_" + str(i), random.uniform, BOUNDS_LOW[i], BOUNDS_HIGH[i])
attributes = ()
for i in range(NUM_OF_PARAMS):
    attributes = attributes + (toolbox.__getattribute__("attribute_" + str(i)),)
toolbox.register("individualCreator", tools.initCycle, creator.Individual, attributes, n=1)
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)
toolbox.register("evaluate", classificationAccuracy)
toolbox.register("select", tools.selTournament, tournsize=2)
toolbox.register("mate", tools.cxSimulatedBinaryBounded, low=BOUNDS_LOW, up=BOUNDS_HIGH, eta=CROWDING_FACTOR)
toolbox.register("mutate", tools.mutPolynomialBounded, low=BOUNDS_LOW, up=BOUNDS_HIGH, eta=CROWDING_FACTOR, indpb=1.0/NUM_OF_PARAMS)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [0]:
#let put the algorithm to run and find the optimal values
population = toolbox.populationCreator(n=POPULATION_SIZE)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("max", numpy.max)
stats.register("avg", numpy.mean)
hof = tools.HallOfFame(HALL_OF_FAME_SIZE)
population, logbook = elitism.eaSimpleWithElitism(population,
                                                  toolbox,
                                                  cxpb=P_CROSSOVER,
                                                  mutpb=P_MUTATION,
                                                  ngen=MAX_GENERATIONS,
                                                  stats=stats,
                                                  halloffame=hof,
                                                  verbose=True)

gen	nevals	max     	avg     
0  	7     	0.934529	0.785937
1  	2     	0.934529	0.927757
2  	4     	0.934529	0.932263
3  	2     	0.934529	0.93379 


In [0]:
#let's print the solution with the higher accuracy
#clarification for the number of layers: 
#(13,) would mean one layer with 14 units, (14, 14) would mean two layer of 
#14 units and (15, 15, 15) would mean three layers of 15 units
print("- Best solution is: \n", test.formatParams(hof.items[0]), "\n 'accuracy' = ", test.getAccuracy(hof.items[0]))1

- Best solution is: 
 'hidden_layer_sizes'=(12,)
 'solver'='sgd'
 'learning_rate_init'='0.019941421593918113'
  
 'accuracy' =  0.9345285714285714


## 4 - Conclusion

###Summary:

####Search Grid: 
Accuracy: 95,18%. Hidden layers: 1. Hidden units: 15. Optimizer: SGD. Learning rate: 0.01.

####Bayes Optimization: 
Accuracy: 94,93%. Hidden layers: 1. Hidden units: 15. Optimizer: SGD. Learning rate: 0.09715.

####Genetic Algorithm: 
Accuracy: 93,45%. Hidden layers: 1. Hidden units: 12. Optimizer: SGD. Learning rate: 0.01994.

###Analysis of results:

Given these results, there are some pretty interest conclusions we can take and some clarifications we must do: In all the cases we get a very high accuracy, over 93%. However, one would expect the highest accuracies to be for the cases with Bayesian Optimization and Genetic Algorithms because thoses two are more sophisticated ones. 

This have a technical reason, when we implemented these algorithms we chose to use very little iterations for the sake of being able of simulate all in the same notebook. If we increase the number of calls in bayer optimization from 12 to 100 this will take a few hours but you will get an accuracy a little bit higher than 95% also. The same happens with the genetic algorithms case, if we increase the number of generations from 3 (this is very poor) to 20 you will get an accuracy higher than 95%. However the accuracy achieved in the Grid Search is very high so it is difficult to surpass.

Finally, we obtained very similar results in the three cases. So, depending on the complexity of the system, the computational power and the design time, any of the three possibilities could be very useful.